**Задание 1**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
import pandas as pd

In [112]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [113]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [114]:
ratings = ratings.groupby(['userId']).agg(['count', 'min', 'max'])

In [119]:
def classify(row):
    return row['timestamp']['max'] - row['timestamp']['min']   

In [120]:
ratings

movieId             rating           timestamp                        
         count  min    max  count  min  max     count         min         max
userId                                                                       
1           20   31   3671     20  1.0  4.0        20  1260759108  1260759205
2           76   10    720     76  1.0  5.0        76   835355395   835356246
3           51   60  84236     51  2.0  5.0        51  1298861589  1298932787
4          204   10   4006    204  1.0  5.0       204   949778714   949982274
5          100    3  48385    100  1.5  5.0       100  1163373044  1163375145
...        ...  ...    ...    ...  ...  ...       ...         ...         ...
667         68    6   1199     68  2.0  5.0        68   847271220   847272234
668         20  296   6425     20  1.0  5.0        20   993613196   993613478
669         37  223   4015     37  2.0  5.0        37  1015829081  1015829766
670         31    1   2912     31  1.0  5.0        31   938781328   940944033
671        115    1   6565    115  1.0  5.0       115  1063500751  1074784735

[671 rows x 9 columns]

In [122]:
ratings['lifetime'] = ratings.apply(classify, axis = 1)
ratings

movieId             rating           timestamp                          \
         count  min    max  count  min  max     count         min         max   
userId                                                                          
1           20   31   3671     20  1.0  4.0        20  1260759108  1260759205   
2           76   10    720     76  1.0  5.0        76   835355395   835356246   
3           51   60  84236     51  2.0  5.0        51  1298861589  1298932787   
4          204   10   4006    204  1.0  5.0       204   949778714   949982274   
5          100    3  48385    100  1.5  5.0       100  1163373044  1163375145   
...        ...  ...    ...    ...  ...  ...       ...         ...         ...   
667         68    6   1199     68  2.0  5.0        68   847271220   847272234   
668         20  296   6425     20  1.0  5.0        20   993613196   993613478   
669         37  223   4015     37  2.0  5.0        37  1015829081  1015829766   
670         31    1   2912     31  1.0  5.0        31   938781328   940944033   
671        115    1   6565    115  1.0  5.0       115  1063500751  1074784735   

          lifetime  
                    
userId              
1             97.0  
2            851.0  
3          71198.0  
4         203560.0  
5           2101.0  
...            ...  
667         1014.0  
668          282.0  
669          685.0  
670      2162705.0  
671     11283984.0  

[671 rows x 10 columns]

In [126]:
ratings[ratings['rating']['count'] > 100]['lifetime'].astype('timedelta64[m]').mean()

Timedelta('8800 days 09:14:22.111164')

In [ ]:
ratings = ratings.groupby(['userId']).agg({'rating': 'count',
                                 'timestamp': lambda x: x.max() - x.min()})

In [89]:
ratings[ratings.rating > 100]['timestamp'].astype('timedelta64[m]').mean()

Timedelta('8800 days 09:14:22.111164')

**Задание 2**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [7]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
len(rzd)

5

In [8]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
len(auto)

5

In [9]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
len(air)

4

In [10]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
len(client_base)

8

In [21]:
rides = rzd.merge(auto, on = 'client_id', how = 'outer').merge(air, on = 'client_id', how = 'outer')
rides

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [26]:
rides.fillna(0, inplace = True) 
rides = rides.astype(int) 

In [27]:
rides

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093,0,0
1,112,2810,0,0
2,113,10283,57483,0
3,114,5774,83,0
4,115,981,912,81
5,116,0,4834,4
6,117,0,98,13
7,118,0,0,173


In [29]:
rides_clients = client_base.merge(rides, on = 'client_id', how = 'inner')
rides_clients

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093,0,0
1,112,Энтузиастов 8а,2810,0,0
2,113,Левобережная 1а,10283,57483,0
3,114,Мира 14,5774,83,0
4,115,ЗЖБИиДК 1,981,912,81
5,116,Строителей 18,0,4834,4
6,117,Панфиловская 33,0,98,13
7,118,Мастеркова 4,0,0,173


**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

можно сделать табличку: визит = 0, покупка = 1
user_id | data | activity | geo_logs
____________________________________
   1      *        0         abcdef
   2      *        1         abcdhj

Гипотезы:
- Если координаты визита и покупки меняются, пользователь тратит деньги в поездках.
- Если у большинства пользователей одни и те же координаты, это самая торговая зона.

3
если набор чисел координат это abcdef, то a определяет самый общий тип, b более локализован, f - самое детализированное.

например a - северное полушарие,  b - россия, с - москва, d - хамовники, e - внутри кольца, d - улица пречистенка, е - здание, f - ресторан дом 13

таким образом сортируя пользователей мы понимаем что 98% пользователей делает покупки в москве, 64% - в центре города, 2% - в конкретном ресторане